多个时间框架
============

依赖于技术分析的最佳交易策略可能会考虑多个时间范围内的价格走势。本教程将展示如何使用backtesting.py来做到这一点，将大部分工作卸载到熊猫重新采样。假设您已经熟悉基本的框架用法。

依靠技术分析的最佳交易策略可能会考虑多个时间框架的价格走势。
本教程将展示如何使用_backtesting.py_,出售的大部分工作
[pandas resampling](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#resampling).
是假定你已经熟悉 [基本框架使用](https://kernc.github.io/backtesting.py/doc/examples/Quick Start User Guide.html).
我们将考验这个做多,应该每年降低 [400%的年交易策略](http://jbmarwood.com/stock-trading-strategy-300/),
使用每天和每周的相对强弱指标 [(RSI)](https://en.wikipedia.org/wiki/Relative_strength_index)
值和移动平均线(MA)。
在实践中，应该使用来自指示符库(如[TA-Lib](https://github.com/mrjbq7/ta-lib) 或 [Tulipy](https://tulipindicators.org))的函数，但在我们中间，让我们介绍将要使用的两个指示符。


In [2]:
import pandas as pd


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # 近似的;足够好
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

策略大致如下:

在以下情况下买入仓位:
* 每周 RSI(30)≥ 每日 RSI(30) > 70
* 收盘价 > MA(10) > MA(2) > MA(50) > MA(100)

在以下情况下平仓:

* 每日收盘价比均线MA(10) 低2%以上
* 8%固定止损被击中

我们需要在最低时间范围（即每日）内提供柱数据，并将其重新采样到我们的策略所需的任何较高时间范围（即每周）。

In [10]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # 每日RSI回顾周期
    w_rsi = 30  # Weekly
    level = 70

    def init(self):
        # 计算策略需求的移动平均
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)

        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)

        # 为了构建每周RSI，我们可以使用库中的' resample_apply() ' helper函数
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)


    def next(self):
        price = self.data.Close[-1]

        # 如果我们还没有持仓，而且所有的条件都满足了，那就进多仓。
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
                self.ma100[-1] < self.ma50[-1] < self.ma20[-1] < self.ma10[-1] < price):


            # 在下次开盘时以市场价买入，但要设置8%的固定止损。
            self.buy(sl=.92 * price)

        # 如果价格以低于10天移动均线2%或更多的价格收盘，如果有的话，要平仓。
        elif price < .98 * self.ma10[-1]:
            self.position.close()

# 让我们看看我们的策略是如何在九年的谷歌股票数据上回测的。
from backtesting.test import GOOG

backtest = Backtest(GOOG, System, commission=.002)
backtest.run()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    2.793296
Equity Final [$]                  10017.44422
Equity Peak [$]                    10978.3801
Return [%]                           0.174442
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                     0.02045
Volatility (Ann.) [%]                4.941212
Sharpe Ratio                         0.004139
Sortino Ratio                         0.00536
Calmar Ratio                         0.002043
Max. Drawdown [%]                   -10.00745
Avg. Drawdown [%]                   -9.340092
Max. Drawdown Duration     2653 days 00:00:00
Avg. Drawdown Duration     1410 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       9.687579
Worst Trade [%]                     -4.456159
Avg. Trade [%]                    

在九年的时间里微薄地进行了四笔交易，回报为零？如果我们稍微优化一下参数呢？

In [5]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

  0%|          | 0/18 [00:00<?, ?it/s]

Wall time: 12.4 s


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   22.486034
Equity Final [$]                  22587.83224
Equity Peak [$]                   23395.59144
Return [%]                         125.878322
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    10.03124
Volatility (Ann.) [%]               13.124247
Sharpe Ratio                         0.764329
Sortino Ratio                         1.28711
Calmar Ratio                         0.530172
Max. Drawdown [%]                  -18.920719
Avg. Drawdown [%]                   -3.795058
Max. Drawdown Duration      778 days 00:00:00
Avg. Drawdown Duration       97 days 00:00:00
# Trades                                   23
Win Rate [%]                        65.217391
Best Trade [%]                      25.034669
Worst Trade [%]                     -6.297769
Avg. Trade [%]                    

In [6]:
backtest.plot()

Row(id='1627', ...)

更好。虽然该策略的表现不如简单的买入并持有，但它的敞口（上市时间）要低得多。

总之，要在多个时间框架上测试策略，您需要在最低时间范围内传递OHLC数据，然后将其重新采样到较高的时间范围，应用指标，然后重新采样回较低的时间范围，填写中间值。这就是函数backtesting.lib.resample_apply（）为你所做的。

通过探索更多示例了解更多信息，或在完整的 API 参考中找到更多框架选项。

Learn more by exploring further
[examples](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#tutorials)
or find more framework options in the
[full API reference](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#header-submodules).